# DAPP on FLamby ISIC2019, Pool Size = 10

In [ ]:
# ============================================================================
# CELL 1: Custom ISIC Dataset Loader
# ============================================================================

import os
from pathlib import Path
from PIL import Image
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset
import torchvision.transforms as transforms


class CustomISIC2019(Dataset):
    def __init__(self, center=0, train=True, data_path="./FLamby/data_isic"):
        self.center    = center
        self.train     = train
        self.data_path = Path(data_path)

        self.image_dir    = self.data_path / "ISIC_2019_Training_Input"
        metadata_file     = self.data_path / "ISIC_2019_Training_Metadata_FL.csv"
        ground_truth_file = self.data_path / "ISIC_2019_Training_GroundTruth.csv"

        metadata     = pd.read_csv(metadata_file)
        ground_truth = pd.read_csv(ground_truth_file)

        unique_datasets = sorted(metadata['dataset'].unique())
        unique_datasets = [d for d in unique_datasets if pd.notna(d)]

        self.dataset_name = unique_datasets[center]
        print(f"  Center {center} = Dataset '{self.dataset_name}'")

        center_metadata = metadata[metadata['dataset'] == self.dataset_name].copy()
        center_data     = center_metadata.merge(ground_truth, on='image', how='inner')

        label_columns = ['MEL', 'NV', 'BCC', 'AK', 'BKL', 'DF', 'VASC', 'SCC']

        def get_label(row):
            for idx, col in enumerate(label_columns):
                if row[col] == 1.0:
                    return idx
            return -1

        center_data['label'] = center_data.apply(get_label, axis=1)
        center_data          = center_data[center_data['label'] >= 0]

        print(f"  Found {len(center_data)} samples")
        for idx, disease in enumerate(label_columns):
            count = (center_data['label'] == idx).sum()
            if count > 0:
                print(f"    {disease}: {count}")

        np.random.seed(42)
        indices   = np.random.permutation(len(center_data))
        split_idx = int(len(center_data) * 0.8)

        selected_indices = indices[:split_idx] if train else indices[split_idx:]
        self.data        = center_data.iloc[selected_indices].reset_index(drop=True)
        print(f"  {'Train' if train else 'Test'} samples: {len(self.data)}")

        if train:
            self.transform = transforms.Compose([
                transforms.Resize((224, 224)),
                transforms.RandomHorizontalFlip(p=0.5),
                transforms.RandomVerticalFlip(p=0.5),
                transforms.RandomRotation(degrees=20),
                transforms.ColorJitter(brightness=0.2, contrast=0.2),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
            ])
        else:
            self.transform = transforms.Compose([
                transforms.Resize((224, 224)),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
            ])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row      = self.data.iloc[idx]
        img_path = self.image_dir / f"{row['image']}.jpg"
        image    = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, int(row['label'])


def get_num_isic_centers(data_path="./FLamby/data_isic"):
    metadata_file   = Path(data_path) / "ISIC_2019_Training_Metadata_FL.csv"
    metadata        = pd.read_csv(metadata_file)
    unique_datasets = metadata['dataset'].unique()
    unique_datasets = [d for d in unique_datasets if pd.notna(d)]
    return len(unique_datasets), unique_datasets


print("✓ CustomISIC2019 dataset loader defined")


# ============================================================================
# CELL 2: Imports + GPU Setup + Logger
# ============================================================================

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import json
import time
from datetime import datetime
from sklearn.metrics import balanced_accuracy_score

# ── GPU Setup ─────────────────────────────────────────────────────────────────
if torch.cuda.is_available():
    device = torch.device('cuda')
    gpu_name = torch.cuda.get_device_name(0)
    gpu_mem  = torch.cuda.get_device_properties(0).total_memory / 1e9
    print("!" * 70)
    print("RUNNING IN GPU MODE")
    print(f"GPU:  {gpu_name}  |  VRAM: {gpu_mem:.1f} GB")
    print("!" * 70)
    torch.backends.cudnn.benchmark     = True
    torch.backends.cudnn.deterministic = False
else:
    device = torch.device('cpu')
    print("WARNING: No GPU found — falling back to CPU")


# ── Experiment Logger ──────────────────────────────────────────────────────────
class ExperimentLogger:
    def __init__(self, experiment_name, log_dir="./experiment_logs"):
        self.experiment_name = experiment_name
        self.log_dir         = Path(log_dir)
        self.log_dir.mkdir(exist_ok=True)
        self.start_time      = datetime.now()
        self.log_file        = (self.log_dir /
            f"{experiment_name}_{self.start_time.strftime('%Y%m%d_%H%M%S')}.json")
        self.log_data = {
            'experiment_name': experiment_name,
            'start_time':      self.start_time.isoformat(),
            'device':          str(device),
            'rounds':          [],
            'final_results':   {},
            'config':          {}
        }
        print(f"✓ Logger: {self.log_file}")

    def log_config(self, config):
        self.log_data['config'] = {k: str(v) for k, v in config.items()}
        self._save()

    def log_round(self, round_num, metrics):
        self.log_data['rounds'].append({
            'round': round_num,
            'timestamp': datetime.now().isoformat(),
            **metrics
        })
        self._save()

    def log_final(self, final_metrics):
        self.log_data['final_results']      = final_metrics
        self.log_data['end_time']           = datetime.now().isoformat()
        self.log_data['total_time_minutes'] = (
            (datetime.now() - self.start_time).total_seconds() / 60
        )
        self._save()
        print(f"✓ Experiment logged: {self.log_file}")

    def _save(self):
        with open(self.log_file, 'w') as f:
            json.dump(self.log_data, f, indent=2)


print("✓ All imports and logger defined")


# ============================================================================
# CELL 3: Install timm
# ============================================================================

try:
    import timm
    print(f"✓ timm: {timm.__version__}")
except ImportError:
    os.system("pip install timm --quiet")
    import timm
    print(f"✓ timm installed: {timm.__version__}")


# ============================================================================
# CELL 4: ViT Backbone
# ============================================================================

class ViTBackbone(nn.Module):
    """ViT-Base/16 pretrained, fully frozen."""
    def __init__(self, pretrained=True):
        super().__init__()
        print(f"\n  Loading ViT-Base/16...")

        vit_with_head        = timm.create_model('vit_base_patch16_224',
                                                  pretrained=pretrained,
                                                  num_classes=21843)
        self.embed_dim       = 768
        self.pretrained_head = None

        if pretrained and hasattr(vit_with_head, 'head'):
            head_layer = vit_with_head.head
            if isinstance(head_layer, nn.Linear):
                self.pretrained_head = {
                    'weight':       head_layer.weight.data.clone(),
                    'bias':         head_layer.bias.data.clone()
                                    if head_layer.bias is not None else None,
                    'in_features':  head_layer.in_features,
                    'out_features': head_layer.out_features
                }
                print(f"  ✓ Pretrained head: [{head_layer.in_features}→{head_layer.out_features}]")

        self.vit = timm.create_model('vit_base_patch16_224',
                                      pretrained=pretrained,
                                      num_classes=0,
                                      global_pool='')
        for param in self.vit.parameters():
            param.requires_grad = False
        self.vit.eval()
        del vit_with_head

        total = sum(p.numel() for p in self.vit.parameters())
        print(f"  ✓ Frozen backbone: {total:,} params | embed_dim={self.embed_dim}")

    def forward(self, x):
        with torch.no_grad():
            features = self.vit.forward_features(x)
            features = features[:, 0]   # CLS token [B, 768]
        return features

    def get_feature_dim(self):          return self.embed_dim
    def has_pretrained_head(self):      return self.pretrained_head is not None
    def get_pretrained_head_weights(self): return self.pretrained_head


print("✓ ViTBackbone defined")


# ============================================================================
# CELL 5: Focal Loss + Class Weight Computer
# ============================================================================

class FocalLoss(nn.Module):
    """
    Focal Loss: FL = -α_t × (1-p_t)^γ × log(p_t)
    α_t = inverse class frequency (rare classes get higher weight)
    γ   = dynamic per client based on rare_ratio
    """
    def __init__(self, alpha=None, gamma=2.0, reduction='mean'):
        super().__init__()
        self.gamma     = gamma
        self.reduction = reduction
        if alpha is not None:
            alpha = alpha.float()
            alpha = alpha / alpha.sum() * len(alpha)
            self.register_buffer('alpha', alpha)
        else:
            self.alpha = None

    def forward(self, logits, targets):
        ce_loss      = F.cross_entropy(logits, targets, reduction='none')
        probs        = F.softmax(logits, dim=1)
        p_t          = probs.gather(1, targets.unsqueeze(1)).squeeze(1)
        focal_factor = (1.0 - p_t) ** self.gamma
        loss         = focal_factor * ce_loss
        if self.alpha is not None:
            loss = self.alpha[targets] * loss
        if self.reduction == 'mean': return loss.mean()
        if self.reduction == 'sum':  return loss.sum()
        return loss


def compute_class_weights(dataset, num_classes, device):
    counts = torch.zeros(num_classes)
    for _, label in dataset:
        counts[int(label)] += 1
    counts  = torch.clamp(counts, min=1.0)
    N       = counts.sum().item()
    weights = N / (num_classes * counts)
    print(f'  Class weights:')
    for k in range(num_classes):
        pct = 100. * counts[k] / N
        print(f'    Class {k}: n={int(counts[k]):5d} ({pct:5.1f}%) → w={weights[k]:.4f}')
    return weights.to(device)


print('✓ FocalLoss + compute_class_weights defined')


# ============================================================================
# CELL 6: Client Descriptor Computer
# ============================================================================

class ClientDescriptorComputer:
    """
    Computes d_n = [μ_n, σ_n, c_n] — 1544-dim statistical descriptor.
    Computed ONCE before federated training, stored on server permanently.
    """
    def __init__(self, backbone, num_classes=8, device='cpu'):
        self.backbone    = backbone
        self.num_classes = num_classes
        self.device      = torch.device(device) if isinstance(device, str) else device
        self.embed_dim   = backbone.get_feature_dim()

    def compute_descriptor(self, train_loader):
        self.backbone.eval()
        all_features = []
        all_labels   = []
        print(f"  Extracting features from {len(train_loader)} batches...")

        with torch.no_grad():
            for images, labels in tqdm(train_loader, desc="    Extracting", leave=False):
                images = images.to(self.device)
                feats  = self.backbone(images)
                all_features.append(feats.cpu())
                all_labels.append(labels.cpu())

        all_features = torch.cat(all_features, dim=0)
        all_labels   = torch.cat(all_labels,   dim=0)

        mu_n    = torch.mean(all_features, dim=0)
        sigma_n = torch.var(all_features,  dim=0, unbiased=True)

        class_counts = torch.zeros(self.num_classes)
        for label in all_labels:
            class_counts[label.item()] += 1
        c_n = class_counts / class_counts.sum()

        descriptor = torch.cat([mu_n, sigma_n, c_n])
        print(f"  ✓ Descriptor shape: {descriptor.shape} | c_n: {c_n.numpy()}")
        return descriptor


print("✓ ClientDescriptorComputer defined")


# ============================================================================
# CELL 7: Prompt Pool
# ============================================================================

class PromptPool(nn.Module):
    """
    Shared prompt pool on the server.
    pool_size=10, can be frozen once all clients converge.
    """
    def __init__(self, pool_size=10, prompt_length=5, embed_dim=768):
        super().__init__()
        self.pool_size     = pool_size
        self.prompt_length = prompt_length
        self.embed_dim     = embed_dim
        self.prompts       = nn.Parameter(
            torch.randn(pool_size, prompt_length, embed_dim) * 0.02
        )
        self._frozen = False
        print(f"  PromptPool: [{pool_size}, {prompt_length}, {embed_dim}] "
              f"| params: {self.prompts.numel():,}")

    def freeze(self):
        """Freeze pool — no more updates after all clients converge."""
        self.prompts.requires_grad = False
        self._frozen = True
        print("  🔒 Prompt pool FROZEN — no further updates")

    def is_frozen(self):
        return self._frozen

    def forward(self):
        return self.prompts


print("✓ PromptPool defined (with freeze support)")


# ============================================================================
# CELL 8: Prompt Selector
# ============================================================================

class PromptSelector(nn.Module):
    """
    Selector W: maps client descriptor → personalized prompt.
    Applies rare disease scaling: α_n' = α_n × (1 + λ·r_n)
    """
    def __init__(self, descriptor_dim=1544, pool_size=10, lambda_rare=0.5):
        super().__init__()
        self.descriptor_dim = descriptor_dim
        self.pool_size      = pool_size
        self.lambda_rare    = lambda_rare
        self.W              = nn.Parameter(
            torch.randn(pool_size, descriptor_dim) * 0.02
        )
        print(f"  PromptSelector: W=[{pool_size}×{descriptor_dim}] | λ={lambda_rare}")

    def compute_rare_class_ratio(self, descriptor):
        class_freqs = descriptor[-8:]
        class_freqs = torch.clamp(class_freqs, min=1e-8)
        entropy     = -(class_freqs * torch.log(class_freqs + 1e-8)).sum()
        max_entropy = np.log(8)
        return 1.0 - (entropy / max_entropy)

    def forward(self, descriptor, prompt_pool):
        if descriptor.device != self.W.device:
            descriptor = descriptor.to(self.W.device)
        logits       = torch.matmul(self.W, descriptor)
        alpha_n      = torch.softmax(logits, dim=0)
        r_n          = self.compute_rare_class_ratio(descriptor)
        scale        = 1.0 + self.lambda_rare * r_n
        alpha_scaled = (alpha_n * scale) / (alpha_n * scale).sum()
        P_n          = torch.einsum('p,plh->lh', alpha_scaled, prompt_pool)
        return P_n, alpha_scaled, r_n


print("✓ PromptSelector defined")


# ============================================================================
# CELL 9: Federated Model
# ============================================================================

class FederatedModelWithPrompts(nn.Module):
    """
    Per-client: frozen ViT + learnable prompt + classifier.
    Classifier initialized from pretrained ViT head.
    """
    def __init__(self, backbone, prompt_length=5, num_classes=8):
        super().__init__()
        self.backbone      = backbone
        self.embed_dim     = backbone.get_feature_dim()
        self.prompt_length = prompt_length
        self.num_classes   = num_classes

        self.prompt = nn.Parameter(
            torch.randn(prompt_length, self.embed_dim) * 0.02
        )
        self.register_buffer('received_prompt', torch.zeros_like(self.prompt.data))

        if backbone.has_pretrained_head():
            pretrained = backbone.get_pretrained_head_weights()
            self.classifier = nn.Sequential(
                nn.LayerNorm(self.embed_dim),
                nn.Linear(self.embed_dim, 512),
                nn.GELU(),
                nn.Dropout(0.1),
                nn.Linear(512, num_classes)
            )
            with torch.no_grad():
                pw = pretrained['weight']
                pb = pretrained['bias']
                self.classifier[1].weight.data = pw[:512, :].clone()
                self.classifier[1].bias.data   = pb[:512].clone()
                self.classifier[4].weight.data = pw[:num_classes, :512].clone()
                self.classifier[4].bias.data   = pb[:num_classes].clone()
            print("  ✓ Classifier init from pretrained ViT head")
        else:
            self.classifier = nn.Sequential(
                nn.LayerNorm(self.embed_dim),
                nn.Linear(self.embed_dim, 512),
                nn.GELU(),
                nn.Dropout(0.1),
                nn.Linear(512, num_classes)
            )

        p_params = self.prompt.numel()
        c_params = sum(p.numel() for p in self.classifier.parameters())
        print(f"  Trainable: prompt={p_params:,} | classifier={c_params:,} | total={p_params+c_params:,}")

    def set_prompts(self, prompts):
        with torch.no_grad():
            self.received_prompt.copy_(prompts.detach())
            self.prompt.copy_(prompts.detach())

    def get_prompts(self):  return self.prompt.detach().clone()
    def get_delta(self):    return (self.prompt.data - self.received_prompt).detach().clone()

    def forward(self, x):
        B        = x.shape[0]
        features = self.backbone(x)
        combined = features + self.prompt.mean(dim=0).unsqueeze(0).expand(B, -1)
        return self.classifier(combined)


print("✓ FederatedModelWithPrompts defined")


# ============================================================================
# CELL 10: Verification
# ============================================================================

print("Verifying pretrained weight initialization...")
_bb = ViTBackbone(pretrained=True)
_m  = FederatedModelWithPrompts(_bb, num_classes=8)
_h  = _bb.get_pretrained_head_weights()
assert torch.allclose(_m.classifier[1].weight.data, _h['weight'][:512, :])
assert torch.allclose(_m.classifier[4].weight.data, _h['weight'][:8, :512])
print("✅ Classifier uses real pretrained ViT weights")
del _bb, _m, _h
if torch.cuda.is_available():
    torch.cuda.empty_cache()


# ============================================================================
# CELL 11: ClientConvergenceTracker
# ============================================================================

class ClientConvergenceTracker:
    """
    Balanced-accuracy-focused convergence tracker.

    Signals (need min_signals_to_stop=2):
      1. Val loss plateau       — no improvement for vl_patience rounds
      2. BalAcc improvement     — best bal_acc not improved for bal_patience rounds
      3. BalAcc regression      — current bal_acc dropped > regression_threshold below best
      4. BalAcc plateau         — variation < bal_acc_threshold for bal_patience rounds

    Imbalance guard:
      Never stop if std_acc - bal_acc > 15%
      (model still majority-class biased)

    Early success:
      If bal_acc >= bal_acc_target for target_patience consecutive rounds → stop
      (client has reached a good balanced accuracy)
    """

    def __init__(self, client_id,
                 # Signal 1 — val loss
                 vl_patience=4,
                 vl_threshold=0.005,
                 # Signals 2,3,4 — all bal_acc based
                 bal_patience=4,
                 bal_acc_threshold=1.5,       # plateau: variation < this %
                 bal_regression_threshold=4.0, # regression: drop > this %
                 # Guard
                 imbalance_gap_threshold=15.0, # never stop if std-bal > this %
                 # Early success
                 bal_acc_target=65.0,          # target balanced accuracy %
                 target_patience=3,            # consecutive rounds at target to stop
                 # General
                 min_rounds=6,
                 min_signals_to_stop=2):

        self.client_id                = client_id
        self.vl_patience              = vl_patience
        self.vl_threshold             = vl_threshold
        self.bal_patience             = bal_patience
        self.bal_acc_threshold        = bal_acc_threshold
        self.bal_regression_threshold = bal_regression_threshold
        self.imbalance_gap_threshold  = imbalance_gap_threshold
        self.bal_acc_target           = bal_acc_target
        self.target_patience          = target_patience
        self.min_rounds               = min_rounds
        self.min_signals_to_stop      = min_signals_to_stop

        self.val_loss_history = []
        self.acc_history      = []
        self.bal_acc_history  = []

        self.best_val_loss    = float('inf')
        self.best_acc         = 0.0
        self.best_bal_acc     = 0.0

        self.stopped             = False
        self.stop_round          = None
        self.stop_reason         = None
        self.rounds_participated = 0

        # Counter for consecutive rounds at/above target
        self._target_hit_count   = 0

    def update(self, val_loss, test_acc, bal_acc):
        self.val_loss_history.append(val_loss)
        self.acc_history.append(test_acc)
        self.bal_acc_history.append(bal_acc)
        self.rounds_participated += 1

        if val_loss < self.best_val_loss: self.best_val_loss = val_loss
        if test_acc > self.best_acc:      self.best_acc      = test_acc
        if bal_acc  > self.best_bal_acc:  self.best_bal_acc  = bal_acc

        # Track consecutive rounds at target
        if bal_acc >= self.bal_acc_target:
            self._target_hit_count += 1
        else:
            self._target_hit_count = 0

    def check_convergence(self, current_round):
        if self.rounds_participated < self.min_rounds:
            return False, None, []

        signals = []
        current_bal = self.bal_acc_history[-1]
        current_std = self.acc_history[-1]

        # ── Early success: bal_acc reached target ──────────────────────
        # If bal_acc has been >= target for target_patience rounds, stop
        if self._target_hit_count >= self.target_patience:
            reason = (f'🎯 BalAcc target reached: '
                      f'{current_bal:.2f}% >= {self.bal_acc_target}% '
                      f'for {self._target_hit_count} consecutive rounds')
            return True, reason, [reason]

        # ── Signal 1: Val loss plateau ─────────────────────────────────
        if len(self.val_loss_history) >= self.vl_patience:
            recent_vl     = self.val_loss_history[-self.vl_patience:]
            best_recent   = min(recent_vl)
            best_previous = (
                min(self.val_loss_history[:-self.vl_patience])
                if len(self.val_loss_history) > self.vl_patience
                else float('inf')
            )
            improvement = best_previous - best_recent
            if improvement < self.vl_threshold:
                signals.append(
                    f'ValLoss plateau (last {self.vl_patience}: '
                    f'{[f"{v:.4f}" for v in recent_vl]}, '
                    f'Δ={improvement:.4f}<{self.vl_threshold})'
                )

        # ── Signal 2: BalAcc not improving ─────────────────────────────
        # Best bal_acc not improved in the last bal_patience rounds
        if len(self.bal_acc_history) >= self.bal_patience:
            recent_bal     = self.bal_acc_history[-self.bal_patience:]
            best_in_window = max(recent_bal)
            # Compare best in recent window vs best before it
            best_before    = (
                max(self.bal_acc_history[:-self.bal_patience])
                if len(self.bal_acc_history) > self.bal_patience
                else 0.0
            )
            bal_improvement = best_in_window - best_before
            if bal_improvement < 0.5:   # less than 0.5% improvement
                signals.append(
                    f'BalAcc not improving (last {self.bal_patience}: '
                    f'{[f"{b:.1f}" for b in recent_bal]}, '
                    f'best_improvement={bal_improvement:.2f}%<0.5%)'
                )

        # ── Signal 3: BalAcc regression ────────────────────────────────
        bal_drop = self.best_bal_acc - current_bal
        if bal_drop > self.bal_regression_threshold and self.best_bal_acc > 0:
            signals.append(
                f'BalAcc regression (best={self.best_bal_acc:.2f}%, '
                f'current={current_bal:.2f}%, '
                f'drop={bal_drop:.2f}%>{self.bal_regression_threshold}%)'
            )

        # ── Signal 4: BalAcc plateau ───────────────────────────────────
        if len(self.bal_acc_history) >= self.bal_patience:
            recent_bal    = self.bal_acc_history[-self.bal_patience:]
            bal_variation = max(recent_bal) - min(recent_bal)
            if bal_variation < self.bal_acc_threshold:
                signals.append(
                    f'BalAcc plateau (last {self.bal_patience}: '
                    f'{[f"{b:.1f}" for b in recent_bal]}, '
                    f'variation={bal_variation:.2f}%<{self.bal_acc_threshold}%)'
                )

        should_stop = len(signals) >= self.min_signals_to_stop

        # ── Imbalance gap guard ────────────────────────────────────────
        # NEVER stop if model is still heavily biased toward majority class
        if should_stop:
            gap = current_std - current_bal
            if gap > self.imbalance_gap_threshold:
                should_stop = False
                signals.append(
                    f'[GUARD] std={current_std:.1f}% bal={current_bal:.1f}% '
                    f'gap={gap:.1f}%>{self.imbalance_gap_threshold}% '
                    f'— still biased, keep training'
                )

        if should_stop:
            return True, ' + '.join(signals), signals
        return False, None, signals

    def get_status_string(self):
        if self.stopped:
            return (f'[STOPPED r{self.stop_round}] '
                    f'BestBal={self.best_bal_acc:.2f}%')
        n = len(self.bal_acc_history)
        if n == 0: return '[not started]'
        gap = self.acc_history[-1] - self.bal_acc_history[-1]
        return (f'[r{n}] '
                f'Bal={self.bal_acc_history[-1]:.2f}% '
                f'(best={self.best_bal_acc:.2f}%) '
                f'VL={self.val_loss_history[-1]:.4f} '
                f'Gap={gap:.1f}% '
                f'TargetHits={self._target_hit_count}')


print('✓ ClientConvergenceTracker v3 — BALANCED ACCURACY FOCUSED')
print('  Signals: ValLoss plateau + BalAcc not improving + BalAcc regression + BalAcc plateau')
print('  Guard:   stop blocked if std-bal gap > 15%')
print('  Bonus:   early stop if bal_acc >= target for 3 consecutive rounds')


# ============================================================================
# CELL 12: Federated Client
# ============================================================================

class FederatedClient:
    def __init__(self, client_id, train_loader, test_loader, backbone,
                 descriptor, num_classes=8, device='cpu'):
        self.client_id     = client_id
        self.train_loader  = train_loader
        self.test_loader   = test_loader
        self.train_dataset = train_loader.dataset
        self.device        = device
        self.descriptor    = descriptor
        self.num_classes   = num_classes
        self.dataset_size  = len(self.train_dataset)

        self.model = FederatedModelWithPrompts(
            backbone=backbone, prompt_length=5, num_classes=num_classes
        ).to(device)

        self.train_losses      = []
        self.train_accs        = []
        self.test_accs         = []
        self.validation_losses = []

        # ── Dynamic Focal Loss gamma from rare_ratio ────────────────────
        # gamma range: [2.0, 5.0] based on client imbalance severity
        print(f'\n  Computing class weights for Client {client_id}...')
        class_freqs   = descriptor[-8:].float()
        class_freqs   = torch.clamp(class_freqs, min=1e-8)
        entropy       = -(class_freqs * torch.log(class_freqs + 1e-8)).sum()
        max_ent       = torch.log(torch.tensor(8.0))
        r_n           = float((1.0 - entropy / max_ent).item())

        # Steeper gamma scaling — harder focus for severely imbalanced clients
        if r_n > 0.6:
            dynamic_gamma = 5.0                  # severely imbalanced (Client 3,4)
        elif r_n > 0.3:
            dynamic_gamma = 2.0 + 3.0 * r_n     # moderate
        else:
            dynamic_gamma = 2.0 + 1.0 * r_n     # balanced

        print(f'  Client {client_id}: rare_ratio={r_n:.3f} → gamma={dynamic_gamma:.2f}')

        alpha = compute_class_weights(train_loader.dataset, num_classes, device)
        self.criterion = FocalLoss(alpha=alpha, gamma=dynamic_gamma, reduction='mean')
        print(f'  ✓ FocalLoss(gamma={dynamic_gamma:.2f}, alpha=inverse_freq)')
        print(f'  Client {client_id}: {self.dataset_size} train samples')

    def set_prompts(self, prompts):  self.model.set_prompts(prompts)
    def get_prompts(self):           return self.model.get_prompts()
    def get_prompt_delta(self):      return self.model.get_delta()

    def train_local(self, base_epochs=4, lr_prompt=0.01,
                    lr_classifier=0.001, max_epochs=10,
                    min_epochs=2, patience=3, threshold=0.001):
        """Local training with patience-based early stopping on val loss."""
        self.model.train()

        optimizer = optim.AdamW([
            {'params': [self.model.prompt],
             'lr': lr_prompt, 'weight_decay': 0.0},
            {'params': self.model.classifier.parameters(),
             'lr': lr_classifier, 'weight_decay': 0.01}
        ])

        use_amp   = (self.device.type == 'cuda')
        scaler    = torch.amp.GradScaler('cuda', enabled=use_amp)
        scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=max_epochs)

        best_val_loss    = float('inf')
        patience_counter = 0
        stop_epoch       = max_epochs

        print(f"\n  Client {self.client_id} | patience={patience}")

        for epoch in range(1, max_epochs + 1):
            ep_loss, ep_acc, _ = self._run_one_epoch(optimizer, scaler, use_amp)
            self.train_losses.append(ep_loss)
            self.train_accs.append(ep_acc)
            scheduler.step()

            val_loss    = self.compute_validation_loss()
            improvement = best_val_loss - val_loss

            if improvement >= threshold:
                best_val_loss    = val_loss
                patience_counter = 0
                status = f"✓ {val_loss:.4f}"
            else:
                patience_counter += 1
                status = f"✗ ({patience_counter}/{patience})"

            print(f"    Ep {epoch}: Loss={ep_loss:.4f} Acc={ep_acc:.2f}% "
                  f"VL={val_loss:.4f} {status}")

            if epoch >= min_epochs and patience_counter >= patience:
                stop_epoch = epoch
                break

        self.validation_losses.append(best_val_loss)
        return self.train_losses[-1], self.train_accs[-1], best_val_loss

    def _run_one_epoch(self, optimizer, scaler, use_amp):
        self.model.train()
        running_loss = correct = total = batch_count = 0

        for images, labels in self.train_loader:
            images = images.to(self.device, non_blocking=True)
            labels = labels.to(self.device, non_blocking=True)
            optimizer.zero_grad()

            with torch.amp.autocast('cuda', enabled=use_amp):
                outputs = self.model(images)
                loss    = self.criterion(outputs, labels)

            scaler.scale(loss).backward()
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=1.0)
            scaler.step(optimizer)
            scaler.update()

            running_loss += loss.item()
            _, predicted  = outputs.max(1)
            total        += labels.size(0)
            correct      += predicted.eq(labels).sum().item()
            batch_count  += 1

        return running_loss / batch_count, 100. * correct / total, batch_count

    def compute_validation_loss(self):
        self.model.eval()
        total_loss = batch_count = 0
        use_amp    = (self.device.type == 'cuda')

        with torch.no_grad():
            for images, labels in self.test_loader:
                images = images.to(self.device, non_blocking=True)
                labels = labels.to(self.device, non_blocking=True)
                with torch.amp.autocast('cuda', enabled=use_amp):
                    outputs = self.model(images)
                    loss    = self.criterion(outputs, labels)
                total_loss  += loss.item()
                batch_count += 1

        return total_loss / max(batch_count, 1)

    def evaluate(self):
        """Returns (std_acc, bal_acc) — primary metric is bal_acc."""
        self.model.eval()
        correct    = total = 0
        all_preds  = []
        all_labels = []
        use_amp    = (self.device.type == 'cuda')

        with torch.no_grad():
            for images, labels in self.test_loader:
                images = images.to(self.device, non_blocking=True)
                labels = labels.to(self.device, non_blocking=True)
                with torch.amp.autocast('cuda', enabled=use_amp):
                    outputs = self.model(images)
                _, predicted = outputs.max(1)
                total       += labels.size(0)
                correct     += predicted.eq(labels).sum().item()
                all_preds.extend(predicted.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        std_acc = 100. * correct / total
        bal_acc = 100. * balanced_accuracy_score(all_labels, all_preds)
        self.test_accs.append(std_acc)
        return std_acc, bal_acc


print("✓ FederatedClient v3 defined")


# ============================================================================
# CELL 13: Federated Server
# ============================================================================

class FederatedServer:
    """
    Federated Server:
    - Prompt pool freezes permanently once all clients converge
    - Convergence tracked via balanced accuracy signals
    - Active clients only receive/send prompts
    """
    def __init__(self, num_classes=8, pool_size=10, prompt_length=5,
                 embed_dim=768, descriptor_dim=1544, device='cpu',
                 lambda_rare=0.5, weight_cap=0.25):   # weight_cap lowered to 0.25

        self.device         = torch.device(device) if isinstance(device, str) else device
        self.num_classes    = num_classes
        self.pool_size      = pool_size
        self.prompt_length  = prompt_length
        self.embed_dim      = embed_dim
        self.descriptor_dim = descriptor_dim
        self.lambda_rare    = lambda_rare
        self.weight_cap     = weight_cap
        self.pool_frozen    = False    # ← NEW: tracks if pool is frozen

        self.prompt_pool = PromptPool(
            pool_size=pool_size, prompt_length=prompt_length, embed_dim=embed_dim
        ).to(device)

        self.prompt_selector = PromptSelector(
            descriptor_dim=descriptor_dim, pool_size=pool_size, lambda_rare=lambda_rare
        ).to(device)

        self.optimizer = optim.Adam(
            list(self.prompt_pool.parameters()) +
            list(self.prompt_selector.parameters()),
            lr=0.001
        )

        self.round_results = []

        total_params = (
            sum(p.numel() for p in self.prompt_pool.parameters()) +
            sum(p.numel() for p in self.prompt_selector.parameters())
        )
        print(f"\n  FederatedServer v3:")
        print(f"    pool_size={pool_size} | prompt_length={prompt_length} | embed_dim={embed_dim}")
        print(f"    weight_cap={weight_cap} (lowered from 0.4) | λ_rare={lambda_rare}")
        print(f"    Total server params: {total_params:,}")
        print(f"    Pool freezes when all clients converge ✓")

    def freeze_prompt_pool(self):
        """
        Permanently freeze prompt pool.
        Called when all clients have converged.
        After this, no more pool or selector updates.
        """
        if not self.pool_frozen:
            self.prompt_pool.freeze()
            self.pool_frozen = True
            print("  🔒 SERVER: Prompt pool permanently frozen")

    def generate_personalized_prompts(self, descriptor):
        with torch.no_grad():
            descriptor   = descriptor.to(self.device)
            prompts_pool = self.prompt_pool()
            P_n, alpha_n, r_n = self.prompt_selector(descriptor, prompts_pool)
        return P_n.cpu().detach().clone(), alpha_n.cpu().detach(), r_n.item()

    def stability_weighted_aggregation(self, client_deltas, validation_losses):
        """
        Capped inverse-loss weighting.
        weight_cap=0.25 prevents any single client dominating.
        """
        print("\n  [Stability-Weighted Aggregation — cap=0.25]")

        val_losses        = np.clip(np.array(validation_losses), 0.001, None)
        inverse_losses    = 1.0 / val_losses
        raw_weights       = inverse_losses / inverse_losses.sum()
        capped            = np.minimum(raw_weights, self.weight_cap)
        stability_weights = capped / capped.sum()

        # Minimum weight floor: no active client gets < 1/(2*n_active)
        n_active  = len(client_deltas)
        min_floor = 1.0 / (2.0 * n_active)
        floored   = np.maximum(stability_weights, min_floor)
        stability_weights = floored / floored.sum()

        print(f"  {'Client':<8} {'VL':>8} {'raw w':>8} {'final w':>8}")
        print(f"  {'─'*40}")
        for i, (vl, rw, cw) in enumerate(zip(val_losses, raw_weights, stability_weights)):
            flag = " ⚠CAP" if rw > self.weight_cap else ""
            print(f"  Client {i:<2}  {vl:>8.4f}  {rw:>8.4f}  {cw:>8.4f}{flag}")

        aggregated_delta = torch.zeros_like(client_deltas[0])
        for delta, weight in zip(client_deltas, stability_weights):
            aggregated_delta += delta.cpu() * weight

        print(f"\n  ✓ ΔP norm: {aggregated_delta.norm().item():.6f}")
        return aggregated_delta, stability_weights

    def update_prompt_pool_with_delta(self, aggregated_delta, alpha_ns, learning_rate=0.01):
        """Only called if pool is NOT frozen."""
        if self.pool_frozen:
            print("  ⏭ Pool frozen — skipping update")
            return
        with torch.no_grad():
            avg_alpha = torch.stack(alpha_ns).mean(dim=0).to(self.device)
            agg_delta = aggregated_delta.to(self.device)
            for p_idx in range(self.pool_size):
                alpha_i = avg_alpha[p_idx].item()
                update  = alpha_i * learning_rate * agg_delta
                self.prompt_pool.prompts.data[p_idx] += update
                if alpha_i > 0.05:
                    print(f"    Slot {p_idx:2d}: α={alpha_i:.3f} ‖upd‖={update.norm().item():.6f}")
        print("  ✓ Pool updated")

    def update_selector_with_delta(self, client_deltas, client_descriptors,
                                   stability_weights):
        """Only called if pool is NOT frozen."""
        if self.pool_frozen:
            print("  ⏭ Pool frozen — skipping selector update")
            return
        self.optimizer.zero_grad()
        losses = []
        for delta, descriptor, weight in zip(
                client_deltas, client_descriptors, stability_weights):
            delta        = delta.to(self.device)
            descriptor   = descriptor.to(self.device)
            P_n, _, _    = self.prompt_selector(descriptor, self.prompt_pool())
            target       = (P_n.detach() + delta).detach()
            losses.append(float(weight) * F.mse_loss(P_n, target))

        total_loss = torch.stack(losses).sum()
        total_loss.backward()
        torch.nn.utils.clip_grad_norm_(self.prompt_selector.parameters(), max_norm=1.0)
        self.optimizer.step()
        print(f"  ✓ Selector updated — loss={total_loss.item():.6f}")

    def federated_round(self, clients, active_mask, trackers, round_num, logger=None):
        """
        Dynamic federated round — balanced accuracy focused.

        Phase 1: Only ACTIVE clients get updated prompts
                 Frozen clients keep last prompt unchanged (pool frozen for them)
        Phase 2: Only active clients train
        Phase 3: Collect deltas from active clients
        Phase 4+5: Aggregate and update ONLY if pool not frozen
        Phase 6: Evaluate ALL clients (primary metric = bal_acc)
        Post:   If all clients converged → freeze pool permanently
        """
        print(f"\n{'='*70}")
        print(f"FEDERATED ROUND {round_num}")
        n_active = sum(active_mask)
        print(f"Active: {n_active}/{len(clients)}  "
              f"IDs={[i for i,a in enumerate(active_mask) if a]}  "
              f"Pool={'FROZEN' if self.pool_frozen else 'active'}")
        print(f"{'='*70}")

        if torch.cuda.is_available():
            print(f"  GPU: {torch.cuda.memory_allocated()/1e9:.2f}GB alloc")

        round_start = time.time()

        # ── Phase 1: Prompts for ACTIVE clients only ────────────────────
        # Frozen clients keep their last received prompt — pool not disturbed
        print("\n[Phase 1] Personalized prompts (active clients only)...")
        alpha_ns    = []
        rare_ratios = []

        for client in clients:
            cid = client.client_id
            if not active_mask[cid]:
                # Frozen client — prompt stays as-is
                r_n = float(self.prompt_selector.compute_rare_class_ratio(
                    client.descriptor.to(self.device)
                ).item())
                print(f"  Client {cid}: [frozen — prompt unchanged] r_n={r_n:.3f}")
                continue

            P_n, alpha_n, r_n = self.generate_personalized_prompts(client.descriptor)
            client.set_prompts(P_n.to(client.device))
            alpha_ns.append(alpha_n)
            rare_ratios.append(r_n)
            rare_tag = "RARE" if r_n > 0.5 else "balanced"
            print(f"  Client {cid}: r_n={r_n:.3f} [{rare_tag}] [active]")

        # ── Phase 2: Local training — active clients only ───────────────
        print("\n[Phase 2] Local training (active clients)...")
        validation_losses  = {}
        active_deltas      = []
        active_descriptors = []
        active_val_losses  = []

        for client in clients:
            cid = client.client_id
            if not active_mask[cid]:
                print(f"  Client {cid}: ⏸ SKIPPED (stopped r{trackers[cid].stop_round})")
                continue
            train_loss, train_acc, val_loss = client.train_local(
                base_epochs=4, lr_prompt=0.01, lr_classifier=0.001
            )
            validation_losses[cid] = val_loss
            print(f"  Client {cid}: TrainAcc={train_acc:.2f}% VL={val_loss:.4f}")

        # ── Phase 3: Collect deltas ─────────────────────────────────────
        print("\n[Phase 3] Collecting ΔP_n (active clients)...")
        for client in clients:
            cid = client.client_id
            if not active_mask[cid]:
                continue
            delta = client.get_prompt_delta().cpu()
            active_deltas.append(delta)
            active_descriptors.append(client.descriptor)
            active_val_losses.append(validation_losses[cid])
            print(f"  Client {cid}: ‖ΔP_n‖={delta.norm().item():.4f}")

        # ── Phase 4+5: Aggregate and update (only if pool not frozen) ───
        if len(active_deltas) > 0 and not self.pool_frozen:
            print("\n[Phase 4] Aggregation...")
            agg_delta, stab_weights = self.stability_weighted_aggregation(
                active_deltas, active_val_losses
            )
            print("\n[Phase 5] Updating pool and selector...")
            self.update_prompt_pool_with_delta(agg_delta, alpha_ns, learning_rate=0.01)
            self.update_selector_with_delta(active_deltas, active_descriptors, stab_weights)
        elif self.pool_frozen:
            print("\n[Phase 4+5] Pool FROZEN — skipping aggregation and update")
            stab_weights = np.array([])
        else:
            print("\n[Phase 4+5] No active clients")
            stab_weights = np.array([])

        if torch.cuda.is_available():
            torch.cuda.empty_cache()

        # ── Phase 6: Evaluate ALL clients ──────────────────────────────
        # Primary metric = balanced accuracy
        print("\n[Phase 6] Evaluation (all clients) — PRIMARY: BalAcc")
        client_accs     = []
        client_bal_accs = []
        newly_stopped   = []

        for client in clients:
            cid              = client.client_id
            std_acc, bal_acc = client.evaluate()
            client_accs.append(std_acc)
            client_bal_accs.append(bal_acc)

            vl = validation_losses.get(
                cid,
                trackers[cid].val_loss_history[-1]
                if trackers[cid].val_loss_history else 0.0
            )
            trackers[cid].update(vl, std_acc, bal_acc)

            if active_mask[cid]:
                should_stop, reason, signals = trackers[cid].check_convergence(round_num)
                if should_stop:
                    active_mask[cid]          = False
                    trackers[cid].stopped     = True
                    trackers[cid].stop_round  = round_num
                    trackers[cid].stop_reason = reason
                    newly_stopped.append(cid)
                    gap = std_acc - bal_acc
                    print(f"  Client {cid}: Bal={bal_acc:.2f}% Std={std_acc:.2f}% "
                          f"Gap={gap:.1f}% ⏹ CONVERGED")
                    print(f"    Reason: {reason[:80]}")
                else:
                    gap     = std_acc - bal_acc
                    sig_str = f'({len(signals)}/2)' if signals else '(0/2)'
                    print(f"  Client {cid}: Bal={bal_acc:.2f}% Std={std_acc:.2f}% "
                          f"Gap={gap:.1f}% ✓ {sig_str}")
            else:
                gap = std_acc - bal_acc
                print(f"  Client {cid}: Bal={bal_acc:.2f}% Std={std_acc:.2f}% "
                      f"Gap={gap:.1f}% ⏸ frozen(r{trackers[cid].stop_round})")

        # ── Freeze pool if ALL clients have now converged ───────────────
        any_active = any(active_mask)
        if not any_active and not self.pool_frozen:
            print(f"\n  All clients converged → FREEZING PROMPT POOL")
            self.freeze_prompt_pool()

        # ── Round summary ────────────────────────────────────────────────
        avg_bal    = np.mean(client_bal_accs)
        avg_acc    = np.mean(client_accs)
        std_dev    = np.std(client_bal_accs)
        elapsed    = time.time() - round_start

        print(f"\n{'─'*70}")
        print(f"ROUND {round_num} SUMMARY  [PRIMARY = Balanced Accuracy]")
        print(f"  Avg BalAcc:  {avg_bal:.2f}%  (std={std_dev:.2f}%)")
        print(f"  Avg StdAcc:  {avg_acc:.2f}%")
        print(f"  Min BalAcc:  {min(client_bal_accs):.2f}%")
        print(f"  Max BalAcc:  {max(client_bal_accs):.2f}%")
        print(f"  Active:      {sum(active_mask)}/{len(clients)}")
        print(f"  Pool:        {'FROZEN 🔒' if self.pool_frozen else 'active'}")
        print(f"  Time:        {elapsed:.0f}s")
        if newly_stopped:
            print(f"  Converged:   clients {newly_stopped}")
        if not any_active:
            print(f"  🎉 ALL CLIENTS CONVERGED")
        print(f"{'─'*70}")

        # Build full weights for logging
        full_weights = np.zeros(len(clients))
        if len(stab_weights) > 0:
            active_ids = [c.client_id for c in clients
                          if c.client_id in validation_losses]
            for idx, cid in enumerate(active_ids):
                if idx < len(stab_weights):
                    full_weights[cid] = stab_weights[idx]

        round_data = {
            'round':             round_num,
            'avg_bal_acc':       avg_bal,        # PRIMARY metric first
            'avg_acc':           avg_acc,
            'std_bal_acc':       std_dev,
            'min_bal_acc':       min(client_bal_accs),
            'max_bal_acc':       max(client_bal_accs),
            'client_accs':       client_accs,
            'client_bal_accs':   client_bal_accs,
            'validation_losses': [
                validation_losses.get(
                    i,
                    trackers[i].val_loss_history[-1]
                    if trackers[i].val_loss_history else 0.0
                )
                for i in range(len(clients))
            ],
            'active_mask':       list(active_mask),
            'n_active':          sum(active_mask),
            'pool_frozen':       self.pool_frozen,
            'stability_weights': full_weights.tolist(),
            'rare_ratios':       rare_ratios,
            'round_time':        elapsed,
            'newly_converged':   newly_stopped
        }
        self.round_results.append(round_data)

        if logger:
            logger.log_round(round_num, round_data)

        return avg_bal, avg_acc, client_bal_accs, client_accs, active_mask, any_active


print("✓ FederatedServer v3 defined")
print("  Pool freezes when all clients converge")
print("  Primary metric: balanced accuracy")
print("  weight_cap=0.25 + min_floor=1/(2*n_active)")


# ============================================================================
# CELL 14: Configuration
# ============================================================================

import platform
safe_workers = 0 if platform.system() == 'Windows' else 4

CONFIG = {
    'num_rounds':     20,        # more rounds — client 4 needs time
    'base_epochs':    5,
    'pool_size':      10,
    'prompt_length':  5,
    'num_classes':    8,
    'lr_prompt':      0.01,
    'lr_classifier':  0.001,
    'batch_size':     64,
    'embed_dim':      768,
    'descriptor_dim': 1544,
    'lambda_rare':    0.5,
    'device':         device,
    'data_path':      './FLamby/data_isic',
    'num_workers':    safe_workers,
    'pin_memory':     True if torch.cuda.is_available() else False,
    # Convergence target (per client)
    'bal_acc_target':  65.0,     # stop early if bal_acc >= this for 3 rounds
    'weight_cap':      0.25,     # lowered from 0.4
}

print("=" * 70)
print("CONFIGURATION v3")
print("=" * 70)
for k, v in CONFIG.items():
    print(f"  {k:22s}: {v}")
print("=" * 70)


# ============================================================================
# CELL 15: Initialize System
# ============================================================================

print("\n" + "=" * 70)
print("INITIALIZING FEDERATED LEARNING SYSTEM v3")
print("=" * 70)

logger = ExperimentLogger(
    experiment_name="pFedPG_v3_BalAcc_FrozenPool",
    log_dir="./experiment_logs"
)
logger.log_config(CONFIG)

# Step 1: Backbone
print("\n[Step 1] Loading ViT backbone...")
backbone = ViTBackbone(pretrained=True).to(device)
print("✓ Backbone ready")

# Step 2: Centers
print("\n[Step 2] Detecting federated centers...")
num_clients, dataset_names = get_num_isic_centers(CONFIG['data_path'])
print(f"✓ {num_clients} centers found")
for i, name in enumerate(dataset_names):
    print(f"  Center {i}: {name}")

# Step 3: Datasets
print("\n[Step 3] Loading ISIC 2019 datasets...")
train_datasets, test_datasets = [], []
for cid in range(num_clients):
    print(f"\n{'─'*50} Client {cid}")
    train_datasets.append(CustomISIC2019(center=cid, train=True,  data_path=CONFIG['data_path']))
    test_datasets.append( CustomISIC2019(center=cid, train=False, data_path=CONFIG['data_path']))
print(f"\n✓ All {num_clients} datasets loaded")

# Step 4: Descriptors
print("\n[Step 4] Computing client descriptors (one-time)...")
descriptor_computer = ClientDescriptorComputer(
    backbone, num_classes=CONFIG['num_classes'], device=device
)
client_descriptors = {}
for cid in range(num_clients):
    print(f"\n{'='*50} Client {cid}")
    desc_loader = DataLoader(
        train_datasets[cid], batch_size=64, shuffle=False,
        num_workers=CONFIG['num_workers'], pin_memory=CONFIG['pin_memory']
    )
    client_descriptors[cid] = descriptor_computer.compute_descriptor(desc_loader)
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
print("\n✓ All descriptors computed")

# Step 5: Clients
print("\n[Step 5] Initializing federated clients...")
clients = []
for cid in range(num_clients):
    print(f"\nClient {cid}...")
    train_loader = DataLoader(
        train_datasets[cid], batch_size=CONFIG['batch_size'], shuffle=True,
        num_workers=CONFIG['num_workers'], pin_memory=CONFIG['pin_memory'],
        persistent_workers=CONFIG['num_workers'] > 0
    )
    test_loader = DataLoader(
        test_datasets[cid], batch_size=CONFIG['batch_size'], shuffle=False,
        num_workers=CONFIG['num_workers'], pin_memory=CONFIG['pin_memory'],
        persistent_workers=CONFIG['num_workers'] > 0
    )
    client = FederatedClient(
        client_id=cid, train_loader=train_loader, test_loader=test_loader,
        backbone=backbone, descriptor=client_descriptors[cid],
        num_classes=CONFIG['num_classes'], device=device
    )
    clients.append(client)
print(f"\n✓ All {num_clients} clients ready")

# Step 6: Server
print("\n[Step 6] Initializing server...")
server = FederatedServer(
    num_classes    = CONFIG['num_classes'],
    pool_size      = CONFIG['pool_size'],
    prompt_length  = CONFIG['prompt_length'],
    embed_dim      = CONFIG['embed_dim'],
    descriptor_dim = CONFIG['descriptor_dim'],
    device         = device,
    lambda_rare    = CONFIG['lambda_rare'],
    weight_cap     = CONFIG['weight_cap']
)
print("✓ Server ready")

print("\n" + "=" * 70)
print("SYSTEM READY")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.memory_allocated()/1e9:.2f} GB used")
print("=" * 70)


# ============================================================================
# CELL 16: Training Loop — Balanced Accuracy Focused
# ============================================================================

print("\n" + "=" * 70)
print("STARTING FEDERATED LEARNING v3 — BALANCED ACCURACY FOCUSED")
print("=" * 70)
print(f"Max rounds:      {CONFIG['num_rounds']}")
print(f"Primary metric:  Balanced Accuracy (NOT standard accuracy)")
print(f"Pool behavior:   Freezes when all clients converge")
print(f"BalAcc target:   {CONFIG['bal_acc_target']}% (early stop if reached for 3 rounds)")
print(f"Convergence:     ≥2 of 4 bal_acc signals per client")
print(f"  Signal 1:      Val loss no improvement (δ<0.005, 4 rounds)")
print(f"  Signal 2:      BalAcc not improving (<0.5% gain, 4 rounds)")
print(f"  Signal 3:      BalAcc regression (>4% drop from best)")
print(f"  Signal 4:      BalAcc plateau (<1.5% variation, 4 rounds)")
print(f"  Guard:         Stop blocked if std-bal gap >15%")
print(f"Weight cap:      {CONFIG['weight_cap']} + min floor=1/(2×n_active)")
print("=" * 70)

# Initialize trackers
trackers = [
    ClientConvergenceTracker(
        client_id                = i,
        vl_patience              = 4,
        vl_threshold             = 0.005,
        bal_patience             = 4,
        bal_acc_threshold        = 1.5,
        bal_regression_threshold = 4.0,
        imbalance_gap_threshold  = 15.0,
        bal_acc_target           = CONFIG['bal_acc_target'],
        target_patience          = 3,
        min_rounds               = 6,
        min_signals_to_stop      = 2
    )
    for i in range(num_clients)
]

active_mask         = [True] * num_clients
best_avg_bal_acc    = 0.0
training_start_time = time.time()

for round_num in range(1, CONFIG['num_rounds'] + 1):

    avg_bal, avg_acc, client_bal_accs, client_accs, active_mask, any_active = \
        server.federated_round(
            clients=clients, active_mask=active_mask,
            trackers=trackers, round_num=round_num, logger=logger
        )

    if avg_bal > best_avg_bal_acc:
        improvement      = avg_bal - best_avg_bal_acc
        best_avg_bal_acc = avg_bal
        print(f"\n🎉 NEW BEST BalAcc: {best_avg_bal_acc:.2f}% (+{improvement:.2f}%)")

    # Print tracker status every 5 rounds
    if round_num % 5 == 0:
        print(f"\n  📊 CLIENT STATUS (Round {round_num}):")
        for tracker in trackers:
            print(f"    Client {tracker.client_id}: {tracker.get_status_string()}")

    if not any_active:
        print(f"\n{'='*70}")
        print(f"ALL CLIENTS CONVERGED — pool frozen, stopping at round {round_num}")
        print(f"{'='*70}")
        break

training_time = time.time() - training_start_time


# ============================================================================
# CELL 17: Final Summary
# ============================================================================

print("\n\n" + "=" * 70)
print("FEDERATED LEARNING COMPLETE v3 — BALANCED ACCURACY REPORT")
print("=" * 70)

final_results = server.round_results[-1]
total_rounds  = len(server.round_results)

print(f"\n📊 FINAL RESULTS [PRIMARY = Balanced Accuracy]")
print(f"{'─'*70}")
print(f"Total rounds:              {total_rounds}/{CONFIG['num_rounds']}")
print(f"Final Avg Balanced Acc:    {final_results['avg_bal_acc']:.2f}%")
print(f"Final Avg Standard Acc:    {final_results['avg_acc']:.2f}%")
print(f"Best Avg Balanced Acc:     {best_avg_bal_acc:.2f}%")
print(f"Pool frozen:               {server.pool_frozen}")
print(f"Total time:                {training_time/60:.1f} min")

print(f"\n📋 PER-CLIENT RESULTS:")
print(f"{'─'*70}")
print(f"{'Client':<8} {'StopRnd':>8} {'BestBal':>9} {'BestStd':>9} "
      f"{'FinalBal':>9} {'Gap':>6} {'Reason'}")
print(f"{'─'*70}")

for i, tracker in enumerate(trackers):
    stop_str  = str(tracker.stop_round) if tracker.stopped else f'>{total_rounds}'
    final_bal = final_results['client_bal_accs'][i]
    final_std = final_results['client_accs'][i]
    gap       = final_std - final_bal
    reason    = (
        tracker.stop_reason[:35] + '...'
        if tracker.stop_reason and len(tracker.stop_reason) > 35
        else tracker.stop_reason or 'still active'
    )
    print(f"Client {tracker.client_id:<2}  "
          f"{stop_str:>8}  "
          f"{tracker.best_bal_acc:>8.2f}%  "
          f"{tracker.best_acc:>8.2f}%  "
          f"{final_bal:>8.2f}%  "
          f"{gap:>5.1f}%  "
          f"{reason}")

print(f"{'─'*70}")
print(f"\nAvg Final BalAcc:  {np.mean(final_results['client_bal_accs']):.2f}%")
print(f"Avg Final StdAcc:  {np.mean(final_results['client_accs']):.2f}%")
print(f"Avg Gap (Std-Bal): {np.mean([a-b for a,b in zip(final_results['client_accs'], final_results['client_bal_accs'])]):.2f}%")

logger.log_final({
    'final_avg_bal_acc':  final_results['avg_bal_acc'],
    'final_avg_acc':      final_results['avg_acc'],
    'best_avg_bal_acc':   best_avg_bal_acc,
    'total_rounds':       total_rounds,
    'pool_frozen':        server.pool_frozen,
    'total_time_minutes': training_time / 60,
    'client_results': [
        {
            'client_id':        t.client_id,
            'stop_round':       t.stop_round,
            'best_bal_acc':     t.best_bal_acc,
            'best_acc':         t.best_acc,
            'stop_reason':      t.stop_reason,
            'bal_acc_history':  t.bal_acc_history,
            'acc_history':      t.acc_history,
            'val_loss_history': t.val_loss_history
        }
        for t in trackers
    ]
})

print(f"\n✓ Results logged to: {logger.log_file}")
print("=" * 70)

✓ CustomISIC2019 dataset loader defined
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
RUNNING IN GPU MODE
GPU:  NVIDIA RTX A6000  |  VRAM: 51.0 GB
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
✓ All imports and logger defined
✓ timm: 0.9.0
✓ ViTBackbone defined
✓ FocalLoss + compute_class_weights defined
✓ ClientDescriptorComputer defined
✓ PromptPool defined (with freeze support)
✓ PromptSelector defined
✓ FederatedModelWithPrompts defined
Verifying pretrained weight initialization...

  Loading ViT-Base/16...
  ✓ Pretrained head: [768→21843]
  ✓ Frozen backbone: 85,798,656 params | embed_dim=768
  ✓ Classifier init from pretrained ViT head
  Trainable: prompt=3,840 | classifier=399,368 | total=403,208
✅ Classifier uses real pretrained ViT weights
✓ ClientConvergenceTracker v3 — BALANCED ACCURACY FOCUSED
  Signals: ValLoss plateau + BalAcc not improving + BalAcc regression + BalAcc plateau
  Guard:   stop blocked if std-bal gap > 15%
 

  ✓ Descriptor shape: torch.Size([1544]) | c_n: [0.22769386 0.341289   0.22809668 0.0593152  0.0918429  0.01007049
 0.00936556 0.03232628]

================================================== Client 1
  Extracting features from 29 batches...


  ✓ Descriptor shape: torch.Size([1544]) | c_n: [0.14886552 0.35583842 0.13115661 0.04482568 0.21748754 0.01438849
 0.00166021 0.08577753]

================================================== Client 2
  Extracting features from 43 batches...


  ✓ Descriptor shape: torch.Size([1544]) | c_n: [0.19962825 0.5505576  0.05873606 0.0063197  0.14200744 0.01486989
 0.02453532 0.00334572]

================================================== Client 3
  Extracting features from 50 batches...


  ✓ Descriptor shape: torch.Size([1544]) | c_n: [6.3231108e-03 9.4119507e-01 3.1615555e-04 0.0000000e+00 3.2247867e-02
 8.2200440e-03 1.1697755e-02 0.0000000e+00]

================================================== Client 4
  Extracting features from 6 batches...


...

  Client 4: Bal=40.83% Std=78.41% Gap=37.6% ✓ (3/2)
  Client 5: Bal=53.84% Std=54.88% Gap=1.0% ⏸ frozen(r7)

──────────────────────────────────────────────────────────────────────
ROUND 8 SUMMARY  [PRIMARY = Balanced Accuracy]
  Avg BalAcc:  55.16%  (std=9.19%)
  Avg StdAcc:  72.34%
  Min BalAcc:  40.83%
  Max BalAcc:  69.14%
  Active:      4/6
  Pool:        active
  Time:        151s
──────────────────────────────────────────────────────────────────────

FEDERATED ROUND 9
Active: 4/6  IDs=[1, 2, 3, 4]  Pool=active
  GPU: 0.79GB alloc

[Phase 1] Personalized prompts (active clients only)...
  Client 0: [frozen — prompt unchanged] r_n=0.217
  Client 1: r_n=0.196 [balanced] [active]
  Client 2: r_n=0.376 [balanced] [active]
  Client 3: r_n=0.859 [RARE] [active]
  Client 4: r_n=0.678 [RARE] [active]
  Client 5: [frozen — prompt unchanged] r_n=0.501

[Phase 2] Local training (active clients)...
  Client 0: ⏸ SKIPPED (stopped r6)

  Client 1 | patience=3
    Ep 1: Loss=0.0207 Acc=75.82% VL

/data2/pobitro/cflenv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:2801: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


  Client 4: Bal=41.99% Std=76.14% Gap=34.1% ✓ (1/2)
  Client 5: Bal=53.84% Std=54.88% Gap=1.0% ⏸ frozen(r7)

──────────────────────────────────────────────────────────────────────
ROUND 10 SUMMARY  [PRIMARY = Balanced Accuracy]
  Avg BalAcc:  57.61%  (std=9.03%)
  Avg StdAcc:  71.24%
  Min BalAcc:  41.99%
  Max BalAcc:  69.14%
  Active:      4/6
  Pool:        active
  Time:        168s
──────────────────────────────────────────────────────────────────────

  📊 CLIENT STATUS (Round 10):
    Client 0: [STOPPED r6] BestBal=69.14%
    Client 1: [r10] Bal=53.15% (best=53.15%) VL=0.0701 Gap=10.6% TargetHits=0
    Client 2: [r10] Bal=62.62% (best=66.56%) VL=0.0760 Gap=11.7% TargetHits=0
    Client 3: [r10] Bal=64.89% (best=66.99%) VL=0.0437 Gap=19.9% TargetHits=0
    Client 4: [r10] Bal=41.99% (best=44.68%) VL=0.1083 Gap=34.1% TargetHits=0
    Client 5: [STOPPED r7] BestBal=59.18%

FEDERATED ROUND 11
Active: 4/6  IDs=[1, 2, 3, 4]  Pool=active
  GPU: 0.79GB alloc

[Phase 1] Personalized promp

/data2/pobitro/cflenv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:2801: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


  Client 4: Bal=43.22% Std=87.50% Gap=44.3% ✓ (3/2)
  Client 5: Bal=53.84% Std=54.88% Gap=1.0% ⏸ frozen(r7)

──────────────────────────────────────────────────────────────────────
ROUND 15 SUMMARY  [PRIMARY = Balanced Accuracy]
  Avg BalAcc:  55.89%  (std=8.99%)
  Avg StdAcc:  74.46%
  Min BalAcc:  43.22%
  Max BalAcc:  69.14%
  Active:      2/6
  Pool:        active
  Time:        103s
──────────────────────────────────────────────────────────────────────

  📊 CLIENT STATUS (Round 15):
    Client 0: [STOPPED r6] BestBal=69.14%
    Client 1: [STOPPED r14] BestBal=53.52%
    Client 2: [STOPPED r11] BestBal=66.56%
    Client 3: [r15] Bal=50.37% (best=66.99%) VL=0.0737 Gap=41.9% TargetHits=0
    Client 4: [r15] Bal=43.22% (best=51.99%) VL=0.1575 Gap=44.3% TargetHits=0
    Client 5: [STOPPED r7] BestBal=59.18%

FEDERATED ROUND 16
Active: 2/6  IDs=[3, 4]  Pool=active
  GPU: 0.79GB alloc

[Phase 1] Personalized prompts (active clients only)...
  Client 0: [frozen — prompt unchanged] r_n=0.21